In [38]:
!pip3 install langchain_community
!pip3 install streamlit plotly


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 9.0 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.1/19.1 MB 26.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 658.1/658.1 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.5/29.5 MB 31.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 29.2 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [15]:

from langchain_community.llms import Ollama

In [16]:
llm = Ollama(model="llama3.2")
llm.invoke("The first man on the moon was ...")


/var/folders/kg/1mj021kj07s5nr3v5_mcnwqc0000gn/T/ipykernel_75926/1637244879.py:1: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama3.2")


'... Neil Armstrong. He stepped onto the lunar surface on July 20, 1969, during the Apollo 11 mission.'

In [17]:
llm.invoke("Can you add an appropriate category next to each of the following expenses. Respond with a list of categories separated by commas. For example, Spotify AB by Adyen - \
Entertainment, Beta Boulders Ams Amsterdam Nld - Sports, etc.: \
Taxi Utrecht, Ministerie van Justitie en Veiligheid, Etos AMSTERDAM NLD, Bistro Bar Amsterdam")

"Here is the list with categories added:\n\nTaxi Utrecht, Health and Wellness, Ministerie van Justitie en Veiligheid, Government, Etos AMSTERDAM NLD, Shopping, Bistro Bar Amsterdam, Dining\n\nNote: Some expenses may fit into multiple categories (e.g. a shopping trip to a restaurant), but I've added the most relevant one based on the context.\n\nHere's a more detailed list with additional categories:\n\nTaxi Utrecht, Transportation, Health and Wellness, Ministerie van Justitie en Veiligheid, Government, Etos AMSTERDAM NLD, Shopping, Dining, Bistro Bar Amsterdam, Food"

In [9]:
import pandas as pd 

df = pd.read_csv('transactions.csv')

In [10]:
display(df)

,Date,Name / Description,Expense/Income,Amount (EUR)
0,2022-01-01 03:25:21.737862,Prescription Meds,Income,181.21
1,2022-01-01 04:19:09.251706,School Supplies,Expense,143.76
2,2022-01-01 09:22:35.753019,Luxury Hotel Stay,Income,331.66
3,2022-01-01 12:37:32.191708,Internet Bill,Income,250.54
4,2022-01-01 14:45:16.644981,Sewer Bill,Income,201.40
...,...,...,...,...
9995,2023-12-30 10:44:08.650836,Student Loan Payment,Income,397.20
9996,2023-12-30 11:04:27.682024,Pet Grooming,Income,72.13
9997,2023-12-30 20:52:12.479587,Trash Collection,Income,343.28
9998,2023-12-30 23:09:24.145395,Legal Fees,Income,24.63


In [11]:
# Get unique transactions in the Name / Description column
unique_transactions = df["Name / Description"].unique()
len(unique_transactions)

191

In [12]:
unique_transactions[1:10]


array(['School Supplies', 'Luxury Hotel Stay', 'Internet Bill',
       'Sewer Bill', 'Home Cleaning', 'Sporting Event',
       'Festival Tickets', 'Loan Repayment', 'Pest Control'], dtype=object)


Categorise bank transactions with Llama2

In [13]:
# Get index list
#https://stackoverflow.com/questions/47518609/for-loop-range-and-interval-how-to-include-last-step
def hop(start, stop, step):
    for i in range(start, stop, step):
        yield i
    yield stop

index_list = list(hop(0, len(unique_transactions), 30))
index_list

[0, 30, 60, 90, 120, 150, 180, 191]

In [33]:
def categorize_transactions(transaction_names, llm):
    # Invoke the LLM to get categorized transactions
    response = llm.invoke(
        "Can you add an appropriate category to the following expenses. "
        "For example: Spotify AB by Adyen - Entertainment, Beta Boulders Ams Amsterdam Nld - Sport, etc. "
        "Categories should be less than 4 words. " + transaction_names
    )
    
    # Split the response by lines
    response_lines = response.split('\n')

    # Filter lines that contain ' - ' (indicating the correct format)
    formatted_lines = [line for line in response_lines if ' - ' in line]

    # Put in dataframe
    categories_df = pd.DataFrame({'Transaction vs category': formatted_lines})
    
    # Split the 'Transaction vs category' column into 'Transaction' and 'Category'
    categories_df[['Transaction', 'Category']] = categories_df['Transaction vs category'].str.split(' - ', expand=True)

    return categories_df


In [34]:
# Test out the function
categorize_transactions('ISS Catering Services De Meern, Taxi Utrecht, Etos AMSTERDAM NLD, Bistro Bar Amsterdam', llm)

,Transaction vs category,Transaction,Category
0,1. Spotify AB by Adyen - Music & Entertainment,1. Spotify AB by Adyen,Music & Entertainment
1,2. Beta Boulders Ams Amsterdam Nld - Outdoor A...,2. Beta Boulders Ams Amsterdam Nld,Outdoor Activities
2,3. ISS Catering Services De Meern - Food & Bev...,3. ISS Catering Services De Meern,Food & Beverage
3,4. Taxi Utrecht - Transportation,4. Taxi Utrecht,Transportation
4,5. Etos AMSTERDAM NLD - Shopping,5. Etos AMSTERDAM NLD,Shopping


In [24]:
# Initialize the categories_df_all dataframe
categories_df_all = pd.DataFrame()

# Loop through the index_list
for i in range(0, len(index_list)-1):
    transaction_names = unique_transactions[index_list[i]:index_list[i+1]]
    transaction_names = ','.join(transaction_names)

    # Call the categorize_transactions function with error handling
    try:
        categories_df = categorize_transactions(transaction_names, llm)
        # Concatenate with the main dataframe
        categories_df_all = pd.concat([categories_df_all, categories_df[['Transaction', 'Category']]], ignore_index=True)
    except ValueError as e:
        print(f"Error during processing batch {i}: {e}")
        continue  # Skip this batch if there's an issue

# Display or save categories_df_all
categories_df_all.to_csv("categorized_transactions.csv", index=False)
print("Categorized transactions saved to 'categorized_transactions.csv'")


Categorized transactions saved to 'categorized_transactions.csv'


In [25]:
display(categories_df_all)

,Transaction,Category
0,1. Prescription Meds,Health
1,2. School Supplies,Education
2,3. Luxury Hotel Stay,Travel
3,4. Internet Bill,Utilities
4,5. Sewer Bill,Utilities
...,...,...
159,7. Book Purchase,Hobbies & Reading
160,8. Hotel Stay,Accommodation
161,9. Debt Collection,Financial Obligations
162,10. Doctor Visit,Health Care


In [26]:
unique_categories = categories_df_all["Category"].unique()
unique_categories

array(['Health', 'Education', 'Travel', 'Utilities', 'Maintenance',
       'Sport', 'Entertainment', 'Finance', 'Food', 'Storage', 'Rent',
       'Shopping', 'Business', 'Romance', 'Charity',
       'Entertainment (Streaming Service)', 'Sport (Fishing License)',
       'Pet Care', 'Food Delivery', 'Legal Expenses',
       'Software/Purchases', 'Relaxation/Leisure', 'Business Expenses',
       'Food and Drink', 'Insurance Premiums', 'Grocery Shopping',
       'Electronics', 'Investments', 'Taxes', 'Health and Wellness',
       'Debt Repayment', 'Home Gym', 'Banking Fees', 'Office Expenses',
       'Yard Maintenance', 'Credit Card Fees', 'Hardware/Paint',
       'Savings', 'Wellness', 'Leisure', 'Gifts', 'Fitness',
       'Subscriptions', 'Moving', 'Healthcare', 'Fuel', 'Transportation',
       'Loans', 'Sports', 'Home Maintenance', 'Business Services',
       'Personal Documents', 'Career Development', 'Side Gig', 'Hobbies',
       'Insurance', 'Gardening', 'Mental Health', 'Banking',
 

In [27]:
# Drop NA values
categories_df_all = categories_df_all.dropna()

# If category contains "Food", then categorise as "Food and Drinks"
categories_df_all.loc[categories_df_all['Category'].str.contains("Food"), 'Category'] = "Food and Drinks"
# If category contains "Clothing", then categorise as "Clothing"
categories_df_all.loc[categories_df_all['Category'].str.contains("Clothing"), 'Category'] = "Clothing"
# If category contains "Services", then categorise as "Services"
categories_df_all.loc[categories_df_all['Category'].str.contains("Services"), 'Category'] = "Services"
# If category contains "Health" or "Wellness", then categorise as "Health and Wellness"
categories_df_all.loc[categories_df_all['Category'].str.contains("Health|Wellness"), 'Category'] = "Health and Wellness"
# If category contains "Sport", then categorise as "Sport
#  and Fitness"
categories_df_all.loc[categories_df_all['Category'].str.contains("Sport"), 'Category'] = "Sport and Fitness"
# If category contains "Travel", then categorise as "Travel"
categories_df_all.loc[categories_df_all['Category'].str.contains("Travel"), 'Category'] = "Travel"

In [36]:
# Remove numbering like "1. " from the Transaction column
categories_df_all['Transaction'] = categories_df_all['Transaction'].str.replace(r'\d+\.\s+', '', regex=True)
display(categories_df_all)

,Transaction,Category
0,Prescription Meds,Health and Wellness
1,School Supplies,Education
2,Luxury Hotel Stay,Travel
3,Internet Bill,Utilities
4,Sewer Bill,Utilities
...,...,...
159,Book Purchase,Hobbies & Reading
160,Hotel Stay,Accommodation
161,Debt Collection,Financial Obligations
162,Doctor Visit,Health and Wellness


In [41]:
# Merge the categories_df_all with the transactions_2022_2023.csv dataframe (df)
df = pd.read_csv("transactions.csv")
df.loc[df['Name / Description'].str.contains("Spotify"), 'Name / Description'] = "Spotify Ab By Adyen"
df = pd.merge(df, categories_df_all, left_on='Name / Description', right_on='Transaction', how='left')
df.to_csv('categorized_transactions.csv')

In [42]:
display(df)

,Date,Name / Description,Expense/Income,Amount (EUR),Transaction,Category
0,2022-01-01 03:25:21.737862,Prescription Meds,Income,181.21,Prescription Meds,Health and Wellness
1,2022-01-01 04:19:09.251706,School Supplies,Expense,143.76,School Supplies,Education
2,2022-01-01 09:22:35.753019,Luxury Hotel Stay,Income,331.66,Luxury Hotel Stay,Travel
3,2022-01-01 12:37:32.191708,Internet Bill,Income,250.54,Internet Bill,Utilities
4,2022-01-01 14:45:16.644981,Sewer Bill,Income,201.40,Sewer Bill,Utilities
...,...,...,...,...,...,...
9995,2023-12-30 10:44:08.650836,Student Loan Payment,Income,397.20,Student Loan Payment,Loans
9996,2023-12-30 11:04:27.682024,Pet Grooming,Income,72.13,Pet Grooming,Pet Care
9997,2023-12-30 20:52:12.479587,Trash Collection,Income,343.28,Trash Collection,Utilities
9998,2023-12-30 23:09:24.145395,Legal Fees,Income,24.63,Legal Fees,Services
